# Quadratic Linearly Constrained Binary Optimization
## Introduction
Quadratic Unconstrained Binary Optimization (Qubo) problems can be formulated from a square, symmetric objective function and a matrix of binary constraints. Suppose we are given an objective function, $O$, of dimension $n \times n$,  and a set of $m$ constraints, represented by the matrix $A$, with dimension $m \times n$ and right-hand side vector $b$ of length $m$. We want to combine them into a Qubo, which can be defined as $Q = O + \alpha (A^T A-2\mathrm{diag}(b^TA))$, where $\alpha \in \mathbb{R}$. At this point, we can find an optimal solution,
$x^{*} = \min_{x} x^T Q x$. 
The parameter $\alpha$ plays an important role in guaranteeing that the constraints are satisfied. We will not go into more detail on this page. 
We will define a simple problem on the Upload tab and show how to upload the components.
Suppose the original problem we want to minimize is 
$-3xy + xz,$
subject to the constraints $x + z = 1$ and  $2x + 2y= 2$.




## Uploading
There are three matrix components that can be associated with to a constraint problem of this type. The objective function in matrix form, the linear constraints matrix, and the right-hand side (RHS) represent the linear constraints themselves. The format should follow the transformation from $\mathbf{A}\vec{x}$ = $\vec{b}$  &rarr;  $\mathbf{A}\vec{x}$ - $\vec{b}$ = 0.


### Uploading and file_id's
First, import the necessary packages:

In [ ]:
import numpy as np
from qci_client import QciClient

In [ ]:
token = "token_here"
api_url = "https://api.qci-prod.com"
qclient = QciClient(api_token=token, url=api_url)

### Formation
For the equation above, $\mathbf{A}\vec{x}$ - $\vec{b}$ = 0, we will break it down into an objective function and constraints.<br>
<br>
&rarr;  $\mathbf{A}$ as the objective function, `obj` <br>
&rarr;  $\vec{b}$ as the constraints (`b` & `rhs`)

In [ ]:
obj = np.array([[ 0. , -1.5,  0.5],
                [-1.5,  0. ,  0. ],
                [ 0.5,  0. ,  0. ]])

The constraints take the form and an explicit RHS vector can be represented as  


In [ ]:
b = np.array([[1, 0, 1],
              [2, 2, 0]])
rhs = -(np.array([[1],
                [2]]))
b, rhs

In [ ]:
constraints = np.hstack((b, rhs))
constraints

Concatenate your objective function and constraints into two dictionaries: 

In [ ]:
qlcbo_obj = {
    'file_name': "smallest_objective.json",
    'file_config': {'objective':{"data": obj, "num_variables": 3}}
}

In [ ]:
qlcbo_constraints = {
    "file_name": "smallest_constraints.json",
    "file_config": {'constraints':{"data": constraints,
    "num_constraints": 2,
    "num_variables": 3}}
}


Now we can upload the various files using the client. Suppose we store the data in a variable data, then we call upload_file to push the data to the server.


In [ ]:
response_json = qclient.upload_file(qlcbo_constraints)
file_id_constraints = response_json["file_id"]

response_json = qclient.upload_file(qlcbo_obj)
file_id_obj = response_json["file_id"]



We can extract the file_id for later use. Triggering a job to run requires the file_id to tell the backend which data to use. We cover this step in the Running Section.
## Running
Running a job involves two key steps to build parameters for the job:
1. Building a job body to submit. 
2. Providing a job_type.

### Building the job_body
The job_body is a dictionary that contains the file_id's and parameter data for running the job.  All job bodies must contain the following data fields, which can be leveraged by the user to track jobs. 



It is easiest to use `qci.build_job_body()` to construct a job_body.

In [ ]:
job_body = qclient.build_job_body(
    job_type="sample-constraint",
    job_params={"nsamples": 40, "alpha": 2, "sampler_type": "dirac-1"},
    constraints_file_id=file_id_constraints,
    objective_file_id=file_id_obj)

In [ ]:
qclient.download_file(file_id=file_id_constraints)


This returns a job_body with the file_id fields appended to the above dictionary. Each of these file_id's was obtained after uploading the corresponding file in the Uploading section. 
Now we can trigger a job using the following command:


In [ ]:
job_response = qclient.process_job(job_body=job_body, job_type="sample-constraint")
job_response


Below we show how to query the result object if an error occurs:

In [ ]:
def error_status(job_response):
    try:
        if job_response['job_info']['details']['status'] == "ERROR":
            return job_response['job_info']['details']['status'], job_response['job_info']['results']['error']
        else:
            return "No errors detected"
    except KeyError:
        return "Error: Unable to retrieve error status information from the job response"

error_status(job_response)